# Carousel mechanism design
Project: *LuSEE-Night* <br>
System: *Antenna Carousel* <br>
Author: *Joe Silber* <br>
Institution: *Lawrence Berkeley National Lab (LBNL)* <br>
Date: *July 2022* <br>

Abstract: Assessment of carousel (a.k.a. "turntable") mechanism and torques for the LuSEE-Night lunar instrument. The radio antennas rotate on a platform. Key design parameters and options for the rotation mechanism are considered here.

## Preliminaries

### Units and coordinate systems
Except where otherwise specified...
- all units are in meters, kilograms, newtons, and seconds
- all inertias and positions are with respect to a global coordinate system, with Z axis concentric to rotation axis

### Python initializations and units

In [1]:
import math
deg = '\u00b0'
Nm_per_ozin = 141.61193227806  # metric/imperial torque conversion

### General material properties

In [2]:
rho_ss = 8000  # stainless steel density
rho_ti = 4430  # titanium density
rho_al = 2700  # aluminum density

### Carbon fiber properties

In [3]:
rho_f = 2170  # carbon fiber density, e.g. CN-80
rho_m = 1170  # resin matrix density, e.g. EX-1515
Mm = 0.32  # matrix mass fraction, i.e. prepreg resin content from vendor
Vv = 0.005  # as-cured void fraction
Vf = (1 - Vv) / (1 + rho_f / rho_m * Mm / (1 - Mm))  # fiber volume fraction, assuming no bleed (conservative)
cfrp_density = rho_f * Vf + rho_m * (1 - Vf - Vv)
print(f'Estimated carbon fiber reinforced plastic (CFRP) density = {cfrp_density:.1f} kg/m^3, volume fraction = {Vf*100:.1f}%')

Estimated carbon fiber reinforced plastic (CFRP) density = 1695.4 kg/m^3, volume fraction = 53.1%


## Key mechanical design criteria
The design criteria below will be used in sizing components and ensuring sufficient mechanical factors of safey. These criteria are informed by key recommendations in NASA's [General Environmental Verification Standards (GEVS)](https://standards.nasa.gov/sites/default/files/standards/GSFC/B/0/gsfc-std-7000b_signature_cycle_04_28_2021_fixed_links.pdf) (accessed 2022-07-09).

### Structural load limit
The limit loads for the carousel mechanism are derived from GEVS Table 2.4-3,
*Generalized Random Vibration Test Levels*. For components weighing less than 22.7 kg, an overall qualification level (in $G_{rms}$) is directly specified. We then apply a proof factor per section 2.4.1.4.1, and a design factor to get the design limit. The design factor accommodated uncertainty in material properties and tolerances in the proof test method. These limits are applied for nominal accelerations in 3 orthogonal axes.

In [4]:
Grms_qual = 14.1
proof_factor = 1.25
design_factor = 1.2
Grms_design = Grms_qual * proof_factor * design_factor
print(f'Design load level = {Grms_design:.1f} G(rms)')

Design load level = 21.1 G(rms)


### Torque margin
Torque margin ($TM$) for the carousel turntable is assessed per GEVS section 2.4.5.3. Two safety factors, $FS_k$ and $FS_v$ are applied to the "known" and "variable" torques in the system. The available driving torque $T_{avail}$ at the motor (not including gearhead) is compared according to:

$$TM = \frac{T_{avail}}{FS_k \sum T_{known} + FS_v \sum T_{variable}} - 1$$

These torques are characterized in GEVS as:

> **Driving Torques**
>
> $T_{avail}$ = Minimum Available Torque or Force generated by the mechanism at worst case environmental conditions at any time in its life. If motors are used in the system, $T_{avail}$ shall be determined at the output of the motor, not including gear heads or gear trains at its output based on minimum supplied motor voltage. $T_{avail}$ similarly applies to other actuators such as springs, pyrotechnics, solenoids, heat actuated devices, etc.
> 
> **Resistive Torques**
>
> $\sum T_{known}$ = Sum of the fixed torques or forces that are known and quantifiable such as
accelerated inertias ($T = I \alpha$) and not influenced by friction, temperature, life, etc. A constant Safety Factor is applied to the calculated torque.
>
> $\sum T_{variable}$ = Sum of the torques or forces that may vary over environmental conditions and life such as static or dynamic friction, alignment effects, latching forces, wire harness
loads, damper drag, variations in lubricant effectiveness, including degradation or depletion of lubricant over life, etc.

Here we use the highest (PDR phase) torque safety factor values from GEVS:

In [5]:
FSk = 2.0  # preliminary design review phase
           # drops to 1.5 at critical design review
           # remains 1.5 at acceptance / qualification testing

FSv = 4.0  # preliminary design review phase
           # drops to 3.0 at critical design review
           # drops to 2.0 at acceptance / qualification testing

### Thermal criteria
The carousel system must survive the lunar day/night cycle. It must operate at intermediate temperatures around dusk/dawn.

The plots below, showing lunar temperature as a function of hour and latitude, were retrieved July 2022 from an article (exact reference?) found through citations [here](https://quickmap.lroc.asu.edu/layers):

![Lunar temperatures as function of hour and latitude](figures/lunar-temperatures.png)

We anticipate LuSEE-Night will land at one of two sites:

1. $155^o$ E and $15^o$ S
2. $175^o$ E and $22^o$ S

These indicate that the lander will experience temperature extrema of:

In [21]:
temp_ranges = {'extrema': [90, 390]}  # Kelvin

Lunar temperature range =  +90 to +390 °K
Lunar temperature range = -183 to +117 °C
Lunar temperature range = -298 to +242 °F


#### Survival temperature range
For design and testing purposes, the survival temperature range is obtained by increasing the expected lunar range by a margin on either end.

In [22]:
temp_margin = 10.0  # deg C
temp_ranges['survival'] = [min(temp_ranges['extrema']) - temp_margin,
                           max(temp_ranges['extrema']) + temp_margin]

Survival temperature range =  +80 to +400 °K
Survival temperature range = -193 to +127 °C
Survival temperature range = -316 to +260 °F


#### Operating temperature range
The operating temperature for carousel rotation will be less extreme. We want a range which is moderate enough to includes practical motors, transmissions, and bearings, yet wide enough to allow a significant period in the dusk/dawn time window where ground-based operators can effectively send commands, monitor the response, and make any adjustments if necessary. A good operating temperature range also offers practical conditions for testing in the lab.

Under the assumption of a perfluoropolyether (PFPE) based lubricant in the gears and bearings ...

? For what temperature range do we have equipment on hand to practically test operating the motors, bearings, transmissions---the full turntable assembly---in vacuum ?

In [ ]:
temp_ranges['operating'] = 

In [ ]:
T_extrema['K'][1] + T_margin]  # Kelvin
T_survival['C'] = [T-273.15 for T in T_survival['K']]  # Celsius
T_survival['F'] = [T*9/5 + 32 for T in T_survival['C']]  # Fahrenheit
for unit, Tminmax in T_survival.items():
    print(f'Survival temperature range = {Tminmax[0]:+4.0f} to {Tminmax[1]:+4.0f} {deg}{unit}')

## Payload
Total mass budget for the carousel system is:

In [7]:
mass_budget_total = 8.0  # kg

Some fraction of this mass (in particular the drive motor and transmission) will be static.

For the purposes of this document, we subdivide the rotating payload into the following estimated masses, moments, and transmission ratios:

### Turntable plate
Three common engineering materials are being considered for the turntable platter: aluminum, titanium, or carbon fiber reinforced plastic (CFRP).

In [8]:
plate_od = 0.500  # outer diameter
plate_id = 0.079  # inner diameter
plate_t = 0.010  # thickness
plate_r2, plate_r1 = plate_od/2, plate_id/2  # radii
plate_area = math.pi * (plate_r2**2 - plate_r1**2)
uniform_plate_inertia_over_mass = 0.5 * (plate_r1**2 + plate_r2**2)

Bending stiffness is achieved either through a honeycomb core or ribs (of which a nominal design might look like that illustrated here:
![Nominal ribs for a turntable](figures/nominal-turntable-ribs.png)

In [9]:
core_vf = 0.15 # honeycomb core volume fraction, typically about 8-32% depending on design
ribs_af = 0.20 # approx area fraction of some nominal ribs stiffening design for turntable plate

#### CFRP sandwich construction
CFRP would have a higher stiffness per unit mass than the metals, as well as some natural structural damping. We would likely use an intermediate modulus fiber in a quasi-isotropic, balanced, symmetric layup. Matrix would be 350$^o$F (450 K) cured resin such as Toray RS-3C, commonly used in spacecraft. The design would need to be sufficiently low-strain such that the total temperature change from processing (450 K) down to lunar night (100 K) does not exceed fiber strain limits. A honeycomb sandwich construction would be most mass-efficient, with titanium or aluminum inserts at bolted connections. Thermal expansion effects would be low, several parts per million (ppm) per degree K. Low out-of-plane thermal conductivity would be of some benefit in reducing the heat transfer rate from the turntable to the rest of the spacecraft below it. The main detractions of CFRP are the higher design and fabrication costs.

In [10]:
plate_sandwich_skin_t = 0.0016
plate_sandwich_core_t = plate_t - 2 * plate_sandwich_skin_t
plate_sandwich_skin_rho = cfrp_density
plate_sandwich_core_rho = rho_al * core_vf
plate_sandwich_mass = (2*plate_sandwich_skin_rho*plate_sandwich_skin_t + plate_sandwich_core_rho*plate_sandwich_core_t) * plate_area
plate_masses = {'CFRP sandwich': plate_sandwich_mass}

#### Metal ribbed plate construction
Material would be either titanium or aluminum.

A machined aluminum plate would be the least expensive to fabricate. Detractions are the higher CTE ($\sim$12-25 ppm/K over the 100K-400K temperature range) and reduced stress allowables at sustained high temperature (7075-T651 aluminum, for example, may have yield strength reduction from $\sim$ 500 MPa at room temperature to $\sim$ 180 MPa when sustained at 150$^o$C (423 K). The high thermal conductivity of aluminum allows faster heat transfer toward the spacecraft. All these detractions can likely be mitigated with use of appropriate design allowables and thermal breaks.

Grade 5 titanium would be a robust solution. Material cost would be comparable to CFRP, though fabrication complexity would be simpler. CTE is relatively low and stable ($\sim$5-9 ppm/K over range 100K-400K), strength is high, and the very low thermal conductivity ($\sim$5-8 W/m*K) would provide some natural insulation of the rest of the spacecraft from the exposed turntable's temperature. For cost and ease of manufacturing we would probably not machine the plate out of thick titanium, but rather screw together a thinner face sheet with stiffening ribs.

In [11]:
plate_rhos = {'aluminum': rho_al,
              'titanium': rho_ti,
             }  # density
plate_face_t = 0.0016  # face plate thickness
plate_ribs_t = plate_t - plate_face_t
plate_volume = (plate_face_t + plate_ribs_t * ribs_af) * plate_area
for name, rho in plate_rhos.items():
    plate_masses[name] = rho * plate_volume

#### Plate construction options summary

In [12]:
plate_inertias = {}
for name, mass in plate_masses.items():
    plate_inertias[name] = mass * uniform_plate_inertia_over_mass  # note this uniform assumption is good for a typical evenly-spaced rib pattern
    print(f'Turntable mass = {plate_masses[name]:.3f} kg, inertia = {plate_inertias[name]:.4f} kg/m^2 ({name})')

Turntable mass = 1.566 kg, inertia = 0.0502 kg/m^2 (CFRP sandwich)
Turntable mass = 1.695 kg, inertia = 0.0543 kg/m^2 (aluminum)
Turntable mass = 2.782 kg, inertia = 0.0891 kg/m^2 (titanium)


In [13]:
# For purposes of motor sizing, select the heaviest option.
plate_mass = max(plate_masses.values())
plate_inertia = max(plate_inertias.values())

### Antenna deployer and pre-amp
Mass and inertia estimates are based on the CAD preliminary design model (July 2022).

#### Stowed configuration:
![Stowed mass and inertia](figures/inertia-deployer-and-preamp-stowed.png)

#### Deployed configuration (3m extension):
![Deployed mass and inertia](figures/inertia-deployer-and-preamp-deployed3m.png)

Note that the masses shown here are not quite identical, due to internal details of the preliminary model. For the conservative purposes of this study, a rough adjustment is made where the larger mass value (stowed) is used to linearly scale up the deployed mass and inertia.

#### (?) Deployed configuration (2m extension):
#### (?) Pattern of 2m vs 3m antennas:

In [14]:
stowed_mass = 3.148
stowed_inertia = 0.1272
deployed_mass_as_modeled = 2.924
deployed_inertia_as_modeled = 4.203
deployed_adjustment = stowed_mass / deployed_mass_as_modeled
deployed_mass = deployed_mass_as_modeled * deployed_adjustment
deployed_inertia = deployed_inertia_as_modeled * deployed_adjustment
print(f'Deployed single antenna + pre-amp (adjusted): mass = {deployed_mass:.3f} kg, inertia = {deployed_inertia:.4} kg/m^2')

Deployed single antenna + pre-amp (adjusted): mass = 3.148 kg, inertia = 4.525 kg/m^2


### Central shaft and bearings
These rotating components are lumped together and approximated as a single tube of metal.

In [15]:
central_od = 0.080  # outer diameter
central_id = 0.070  # inner diameter
central_length = 0.020  # length
central_r1, central_r2 = central_id/2, central_od/2  # radii
central_rho = rho_ss
central_mass = central_rho * central_length * math.pi * (central_r2**2 - central_r1**2)
central_inertia = 0.5 * central_mass * (central_r1**2 + central_r2**2)
print(f'Central shaft and bearing: mass = {central_mass:.3f} kg, inertia = {central_inertia:.4} kg/m^2')

Central shaft and bearing: mass = 0.188 kg, inertia = 0.0002662 kg/m^2


### Parallel-axis transmission

### Motor shaft, gears, bearing
In the present study (July 2022), whose initial purpose is to define specifications for the motor, the internal details of the motor are not considered. They could be added to this study at a later date, after we have selected a specific motor model.

## Friction

### External dust seal

Some dust seal(s) will be required to prevent lunar dust from accessing the bearing and transmission. These impose some friction load on the assembly.

Here I assume the seal contact is made against a flat face by annular, thin-walled polyimide. In this approach, there would be one seal at the turntable rotation axis, and another at the motor output, covering the off-axis transmission components. (The motor may additionally have internal seals in its bearing at the output shaft, TBD.)

The seal at the central axis will bear either on the rotating turntable, or else be mounted to the turntable and bear on its opposing counterpart (i.e. the top plate or the central shaft support). Preload force will be relatively low, since only dust needs to be excluded.

Some comments are offered in another section about the option of alternatively, or additionally, having internal seals integral to the bearing. There are both advantages and disadvantages. It would be necessary for the bearing vendor to fabricate using a seal material with which we are comfortable at the lunar temperature extremes and in vacuum.

In [16]:
seal_contact_radius = 0.080  # annulus of contact for the seals
seal_contact_width = 0.003  # rough guess as to desirable contact width, will depend on seal design details
seal_contact_pressure_millibar = 10  # rough guess as to desirable clamping pressure (per seal)
seal_contact_pressure = seal_contact_pressure_millibar * 100
seal_contact_area = 2 * math.pi * seal_contact_radius * seal_contact_width
seal_clamping_force = seal_contact_pressure * seal_contact_area
print(f'External dust seal at radius {seal_contact_radius:.3f} m: clamping force = {seal_clamping_force:.1f} N')

External dust seal at radius 0.080 m: clamping force = 1.5 N


Datasheet values for the static and dynamic friction coefficient of [Vespel SP-1](https://www.curbellplastics.com/Research-Solutions/DuPont-Vespel-SP-1) are listed at 0.29 and 0.35, respectively. I presume a fixed, static value of 0.35 for the purposes of this calculation, and estimate a fixed torque based on the assumed contact annulus.

In [17]:
seal_mu = 0.35
seal_torque = seal_mu * seal_contact_radius * seal_clamping_force
print(f'External dust seal static torque = {seal_torque:.3f} N*m = {seal_torque * Nm_per_ozin:.1f} oz*in')

External dust seal static torque = 0.042 N*m = 6.0 oz*in


### Bearing friction
For the purposes of preliminary overall system torque calculations, I presume a bearing with its own integrated dust seal. This may not be the case, as discussed in more detail in the bearing selection section below. But presently, in sizing the motor and transmission, I assume selection of a single Type-X, nitrile double-sealed, stainless steel bearing, e.g. Kaydon WA025XP0. The vendor provides a static torque value for this bearing, given default lubrication and temperature conditions:

In [18]:
bearing_torque_ozin = 8.0  # oz*in
bearing_torque = bearing_torque_ozin / Nm_per_ozin
print(f'Central bearing static torque = {bearing_torque:.3f} N*m')

Central bearing static torque = 0.056 N*m


In [19]:



payloads = {'motor internal':
                {'mass': 0.0,
                 'inertia': 0.0,
                 'count': 1,
                 'ratio': 1.0,
                },
            
            'motor external':
                {'mass': 0.0,
                 'inertia': 0.0,
                 'count': 1,
                 'ratio': 1.0,
                },
            
            'main shaft bearing':
                {'mass': 0.0,
                 'inertia': 0.0,
                 'count': 1,
                 'ratio': 1.0,
                },            
            
            'main shaft':
                {'mass': 0.0,
                 'inertia': 0.0,
                 'count': 1,
                 'ratio': 1.0,
                },    
            
            'plate':
                {'mass': plate_mass,
                 'inertia': 0.0,
                 'count': 1,
                 'ratio': 1.0,
                },
            
            'antenna (deployed)':
                {'mass': 0.0,
                 'inertia': 0.0,
                 'count': 4,
                 'ratio': 1.0,
                },
           }

## Bearings

### Bearing type and size
The preliminary design (July 2022) packaging acommodates a pair of Kaydon KA025AR0 bearings:

![Kaydon KA0xxAR0](figures/kaydon-KA0xxAR0-open-A-bearings.png)

This is a thin section, angular contact, open bearing, with bore 2.5\" (63.5 mm), O.D. 3.0\" (76.2 mm), and thickness 0.25\" (6.35 mm). Mass is low, ~0.12 lbs = 55 g. We utilize the common design pattern of mounting in pairs, to provide tilt resistance.

Stainless steel options (may have custom lead-times and will be more expensive) are available. Replace "K" with "S" in the part number.

Kaydon also offers an attractive "Type X" four-point contact bearing. We have previously used such a bearing (at a much larger scale) in the Dark Energy Spectroscopic Instrument (DESI) ADC rotator mechanism. The type X controls 5 degrees of freedom in a single unit, rather than the typical angular contact duplex. This has the advantage of fewer parts and less weight, as well as removing the need to control thrust preload in the mounting between the two separate bearing units. A type X unit of similar package size as KA025AR0 would be KA025XP0:

![Kaydon KA0xxXP0](figures/kaydon-KA0xxXP0-open-X-bearings.png)

Again, replace "K" with "S" for equivalent stainless option.

### External vs internal bearing seals

For these open bearings, we would make a dust seal that rides on the shaft and/or flat face. Call this an "external" seal. It has the advantages of being:

- independent from the bearing selection and procurement
- easily replaceable
- visible and inspectable
- material of our choice (i.e. robust, low cte, temperature tolerant material like polyimide)

We can alternatively purchase a bearing with integrated seals. Call this an "internal" seal. It has the advantages of:

- lower part count
- lower mass
- more compact packaging

Again taking Kaydon as an example bearing manufacturer, their standard seal material is nitrile rubber. This would need to be qualified for our temperature range (100 K - 400 K). Kaydon does offer customization options in alternate seal materials, but we have not yet checked whether polyimide in particular would work for them. Several models Kaydon offers with an integrated double-seal are given in this table:

![Kaydon JA0xxXP0](figures/kaydon-JA0xxXP0-sealed-X-bearings.png)

The Kaydon line-up does not have sealed bearings in angular contact configurations. 

For sealed bearings, replace "J" with "W" in the part number for the stainless option.

### Bearing seal materials

If we fabricate our own seals, one might consider using [Vespel SP-3](https://www.curbellplastics.com/Research-Solutions/DuPont-Vespel-SP-3). This is a 15% MoS<sub>2</sub>-filled polyimide with a long history of use in space applications, with low creep, high strength, and low coefficient of thermal expansion (for a polymer), and which should survive the temperature extremes.

A concern I have is that the friction coefficient of SP-3 in vacuum is reported to be very low (0.03) in the Dupont datasheet. Normally lower friction is "good" in a bearing, however in our design, having some consistent friction is helpful, to guarantee stability of the position of the turntable stable when powered off. So we probably rather prefer having a consistent friction whether testing in lab air or operating in vacuum. In this respect, unfilled [Vespel SP-1](https://www.curbellplastics.com/Research-Solutions/DuPont-Vespel-SP-1) may be a simpler and better choice. However, the DuPont datasheet does not specifically list a vacuum friction coefficient, so this bears some further investigation.

Graphite-filled polyimide grades would not be chosen, since the graphite lubricant can in fact become abrasive in a moisture-free environment.

### Turntable bearings

There are some bearings available with more integrated turntable functionality, for example including an external gear profile. An attractive option may be Kaydon T01-00325EAA:

![Kaydon turntables - basic](figures/kaydon-turntable-bearings-basic.png)

![Kaydon turntables - external gears](figures/kaydon-turntable-bearings-extgear.png)

![Kaydon turntables - illustrations](figures/kaydon-turntable-illustrations.png)

These turntables are not off-the-shelf parts. Cost and lead-time would be TBD. Performance is not particularly any better than the other options; the attraction here would be the compact packaging. The gearing would need some exterior protection from lunar dust.

## Harness
The primary elastic contributor to torque is the cable harness. Its torsional stiffness is estimated here by counting the number of conductors, guessing a gauge and strand count for each conductor, and then adding up their torsional stiffness plus their plastic jackets in parallel.

## References
- [NASA General Environmental Verification Standards (GEVS)](https://standards.nasa.gov/sites/default/files/standards/GSFC/B/0/gsfc-std-7000b_signature_cycle_04_28_2021_fixed_links.pdf)
- [Kaydon turntable bearings](https://www.kaydonbearings.com/RealiSlim_TT_bearings.htm)
- [Kaydon sealed, slim bearings](https://www.kaydonbearings.com/RealiSlim_sealed_bearings.htm)